In [ ]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
from google.colab.patches import cv2_imshow
import cv2 as cv
import numpy as np
import random

In [ ]:
# Reading the dataset
# 2 modes: 'file' or 'folder'
def read_images(dataset_path,mode, batch_size):
    data = []
    imagepaths, labels = list(), list()
    if mode == 'file':
        # Read dataset file
        data = open(dataset_path, 'r').read().splitlines()
        for d in data:
            imagepaths.append(d.split(' ')[0])
            labels.append(int(d.split(' ')[1]))
    elif mode == 'folder':
        # An ID will be affected to each sub-folders by alphabetical order
        label = 0
        # List the directory
        try:  # Python 2
            classes = sorted(os.walk(dataset_path).next()[1])
        except Exception:  # Python 3
            classes = sorted(os.walk(dataset_path).__next__()[1])
        # List each sub-directory (the classes)
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            try:  # Python 2
                walk = os.walk(c_dir).next()
            except Exception:  # Python 3
                walk = os.walk(c_dir).__next__()
            # Add each image to the training set
            for sample in walk[2]:
                # Only keeps jpeg images
                if sample.endswith('.jpg') or sample.endswith('.jpeg'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")

    for img,lab_ in zip(imagepaths,labels):
      array = cv.imread(img)
      if array.shape==(480,640,3):
        data.append((array, lab_))
        print("*",end='')
    
    return data


learning_rate = 0.001
num_steps = 10000
batch_size = 128
display_step = 100

# Network Parameters
dropout = 0.75 # Dropout, probability to keep units

DATASET_PATH = '/content/gdrive/My Drive/dataset'
MODE = 'folder'
batch_size  = 32
# Build the data input
dataset = read_images(DATASET_PATH, MODE, batch_size)

In [ ]:
#run this to see if you have loaded the dataset correctly
test = dataset[500]
cv2_imshow(test[0])
print(test[1])

In [ ]:
# you can save your dataset for later use and load it using the next block
with open('/content/gdrive/My Drive/dataset/test.npy', 'wb') as f:
    np.save(f, dataset)


In [ ]:
#load an already saved dataset(np.ndarray)
dataset = np.load('/content/gdrive/My Drive/dataset/test.npy',allow_pickle=True)


In [ ]:
def split_data_label(in_list):
  data = []
  label = []

  for item in in_list:
    data.append(np.array(item[0], dtype='float32'))
    label.append(item[1])
  
  return np.array(data), np.array(label)


In [ ]:
import copy 

dataset_copy = copy.deepcopy(dataset)
np.random.shuffle(dataset_copy)
np.random.shuffle(dataset_copy)
np.random.shuffle(dataset_copy)

print(type(dataset_copy))
print(type(dataset))

print(dataset[0])
print(dataset_copy[0])

split_index = round(len(dataset_copy)*0.8)
t_data = dataset_copy[:split_index]
v_data = dataset_copy[split_index:]
print(split_index)

In [ ]:
#run this to check if the dataset has shuffled
cv2_imshow(dataset[1000][0])

In [ ]:
cv2_imshow(dataset_copy[1000][0])

In [ ]:
# split the train and test data
df_train, y_train = split_data_label(t_data)
df_test, y_test = split_data_label(v_data)

In [ ]:
print(df_train[0].shape)
img_height = df_train[0].shape[0]
img_width = df_train[0].shape[1]

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
                             tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
                             tf.keras.layers.MaxPooling2D(),
                             tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
                             tf.keras.layers.MaxPooling2D(),
                             tf.keras.layers.Conv2D(96, 3, padding='same', activation='relu'),
                             tf.keras.layers.MaxPooling2D(),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(10, activation='relu'),
                             tf.keras.layers.Dense(5)
])


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# used for debugging.Gives you the type of inputs and their shapes
[print(i.shape, i.dtype) for i in model.inputs]

In [ ]:
model.fit(df_train, y_train, epochs=10)

In [ ]:
model.evaluate(df_test, y_test)

In [ ]:
predictions = model.predict(df_test)

In [ ]:
index = 82
x = np.array([df_test[index]])
model.predict(x, batch_size=1)
print(np.argmax(model.predict(x, batch_size=1)))
print(" vs ")
print(y_test[index])
cv2_imshow(df_test[index])


In [ ]:
index = 20
classes = ['eating','praying','sleeping','studying']
print(classes[int(y_test[index])])
print(classes[np.argmax(predictions[index])])
cv2_imshow(df_test[index]*255)

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
# saves your model in the drive
model.save('/content/gdrive/My Drive/dataset/new_recognizerv3_80.h5')